In [18]:
nautilus_editors_note = """Behold the humble nautilus. Just about a foot in diameter, it is a slow bottom-dweller with short tentacles that moves through the water with an unsteady wobble. It’s also 500 million years old and, in its day, was the best and brightest, using its newly evolved depth control to lay waste to acre after acre of scuttling crustacean prey.',
 'We became interested in it here at Nautilus because, well, we stole its name. But also because (for a mollusk) it represents a remarkable intersection of science, math, myth, and culture. Since that is exactly the kind of intersection we love to write about, we decided to put together a little “teaser” issue all about it.',
 'There’s the science. The nautilus has a beautiful, logarithmic, and fractal spiral in its shell. Benoit Mandelbrot, discoverer of the fractal, gives us a few words on that topic. One of the world’s foremost nautilus experts, Peter Ward, tells us about nautilus evolution and biology, and about his life of nautilus research.',
 'Then, the myth: from Jules Verne’s fictional submarine, to Oliver Wendell Holmes’ poem, to how and why we turn science into story.',
 'Two chapters, one undersea creature. Welcome aboard."""

# apply ftfy?
nautilus_editors_note = """'Behold the humble nautilus. Just about a foot in diameter, it is a slow bottom-dweller with short tentacles that moves through the water with an unsteady wobble. It\'s also 500 million years old and, in its day, was the best and brightest, using its newly evolved depth control to lay waste to acre after acre of scuttling crustacean prey.\',\n \'We became interested in it here at Nautilus because, well, we stole its name. But also because (for a mollusk) it represents a remarkable intersection of science, math, myth, and culture. Since that is exactly the kind of intersection we love to write about, we decided to put together a little "teaser" issue all about it.\',\n \'There\'s the science. The nautilus has a beautiful, logarithmic, and fractal spiral in its shell. Benoit Mandelbrot, discoverer of the fractal, gives us a few words on that topic. One of the world\'s foremost nautilus experts, Peter Ward, tells us about nautilus evolution and biology, and about his life of nautilus research.\',\n \'Then, the myth: from Jules Verne\'s fictional submarine, to Oliver Wendell Holmes\' poem, to how and why we turn science into story.\',\n \'Two chapters, one undersea creature. Welcome aboard.'"""

In [12]:
import ftfy

[e.strip() for e in ftfy.ftfy(nautilus_editors_note).split('\n')]

["Behold the humble nautilus. Just about a foot in diameter, it is a slow bottom-dweller with short tentacles that moves through the water with an unsteady wobble. It's also 500 million years old and, in its day, was the best and brightest, using its newly evolved depth control to lay waste to acre after acre of scuttling crustacean prey.',",
 '\'We became interested in it here at Nautilus because, well, we stole its name. But also because (for a mollusk) it represents a remarkable intersection of science, math, myth, and culture. Since that is exactly the kind of intersection we love to write about, we decided to put together a little "teaser" issue all about it.\',',
 "'There's the science. The nautilus has a beautiful, logarithmic, and fractal spiral in its shell. Benoit Mandelbrot, discoverer of the fractal, gives us a few words on that topic. One of the world's foremost nautilus experts, Peter Ward, tells us about nautilus evolution and biology, and about his life of nautilus rese

## Cocqui OSS

In [36]:
from pathlib import Path
import pandas as pd
from TTS.api import TTS


model_names = [
    'tts_models/en/ek1/tacotron2',
 'tts_models/en/ljspeech/tacotron2-DDC',
 'tts_models/en/ljspeech/tacotron2-DDC_ph',
 'tts_models/en/ljspeech/glow-tts',
]

model_names = [e for e in TTS.list_models() if 'en' in e]
save_dir = Path("./data/tts_output/dev/")
if not save_dir.exists():
    save_dir.mkdir(parents=True, exist_ok=True)

for model_name in model_names:
    try:
        tts = TTS(model_name=model_name, gpu=True)
        file_save = save_dir / f"nautilus_{Path(model_name).name}.wav"
        if file_save.exists:
            continue
        tts.tts_to_file(
            text=nautilus_editors_note, file_path=file_save
        )
    except Exception:
        print(f"**Unable to use {model_name}")
        continue


## AWS Polly TTS

In [ ]:
import boto3

# AU polly voice models
model_names = [
    "Nicole",
    "Russell",
]

# Create a client using your credentials
polly_client = boto3.Session().client("polly")

for model_name in model_names:
    print(model_name)
    # Use Polly
    response = polly_client.synthesize_speech(
        VoiceId=model_name, OutputFormat="mp3", Text=nautilus_editors_note
    )

    # Save the audio to a file
    file_save = save_dir / f"nautilus_{Path(model_name).name}.wav"
    with open(str(file_save), "wb") as file:
        file.write(response["AudioStream"].read())
